In [1]:
from math import *
import numpy as np
import pandas as pd
from datetime import datetime

import skyfield
from skyfield.api import load
from skyfield.api import N, W, S, E, wgs84
from skyfield.api import Star
from skyfield.data import hipparcos
from skyfield.almanac import find_discrete, risings_and_settings

# Skyfield Globals

In [2]:
ts = load.timescale()
planets = load('de421.bsp')
earth = planets['earth']

with load.open(hipparcos.URL) as f:
    stars_df = hipparcos.load_dataframe(f)
    
star_dictionary = {"Alpheratz":677, "Ankaa":2081, "Schedar":3179, "Diphda":3419, "Achernar":7588, "Hamal":9884, "Polaris":11767, "Acamar":13847, "Menkar":14135, "Mirfak":15863, "Aldebaran":21421, "Rigel":24436, "Capella":24608, "Bellatrix":25336, "Elnath":25428, "Alnilam":26311, "Betelgeuse":27989, "Canopus":30438, "Sirius":32349, "Adhara":33579, "Procyon":37279, "Pollux":37826, "Avior":41037, "Suhail":44816, "Miaplacidus":45238, "Alphard":46390, "Regulus":49669, "Dubhe":54061, "Denebola":57632, "Gienah":59803, "Acrux":60718, "Gacrux":61084, "Alioth":62956, "Spica":65474, "Alkaid":67301, "Hadar":68702, "Menkent":68933, "Arcturus":69673, "Rigil Kent.":71683, "Kochab":72607, "Zuben'ubi":72622, "Alphecca":76267, "Antares":80763, "Atria":82273, "Sabik":84012, "Shaula":85927, "Rasalhague":86032, "Eltanin":87833, "Kaus Aust.":90185, "Vega":91262, "Nunki":92855, "Altair":97649, "Peacock":100751, "Deneb":102098, "Enif":107315, "Al Na'ir":109268, "Fomalhaut":113368, "Scheat":113881, "Markab":113963}

In [3]:
ts

# Utility Functions

In [4]:
# Mind blown... python uses Bakers Rounding.  Got to resort to this to get proper rounding.
from decimal import *
getcontext().rounding = ROUND_HALF_UP

def normal_round(value, precision):
    value = value * 10**precision
    value = Decimal(value).to_integral_value()
    value = value / 10**precision
    return float(value)

def createAngle(degrees, minutes, sign):
    if (sign == None): 
        sign = 1
    return (degrees + minutes/60.0) * sign

def toDegreesAndMinutes(angle):
    sign = 1
    if (angle < 0): sign = -1
    angle = angle*sign
    degrees = floor(angle)
    minutes = normal_round((angle - degrees) * 60.0, 2)
    
    return degrees*sign, minutes
    
def angleToString(angle):
    degrees, minutes = toDegreesAndMinutes(angle)
    return "{} degrees; {} minutes".format(degrees, minutes)

def angleToStringDelta(angle):
    degrees, minutes = toDegreesAndMinutes(angle)
    if (degrees == 0):
        return "{} minutes".format(minutes)
    else:
        return "{} degrees; {} minutes".format(degrees, minutes)
    
def diffAngle (angle1, angle2, zero_threshold):
    threesixty_threshold = 360-zero_threshold
    
    if ((angle1 < zero_threshold) and (angle2 > threesixty_threshold)):
        angle1 += 360 # Move up Angle 1
    elif((angle2 < zero_threshold) and (angle1 > threesixty_threshold)):
        angle2 += 360 # Move up Angle 2
    
    return abs (angle1-angle2)

# Test Data

In [5]:
def readAngle(df, index, attribute):
    value = df.loc[index][attribute]
    degrees, minutes, desc = value.split(':')
    sign = 1 # assume positive sign.
    
    if (desc == 'N'):
        sign = N
    elif(desc == 'S'):
        sign = S
    elif(desc == 'E'):
        sign = E
    elif(desc == 'W'):
        sign = W
    elif(desc == 'On'):
        sign = On
    elif(desc == 'Off'):
        sign = Off
    return createAngle(float(degrees), float(minutes), sign)

def read_lop_input (df, index):
    celestial_object = df.loc[index]['celestial_object']
    utc = df.loc[index]['utc']
    dr_lat = readAngle(df, index, 'dr_lat')
    dr_lon = readAngle(df, index, 'dr_lon')
    Hs = readAngle(df, index, 'Hs')
    ic = readAngle(df, index, 'ic')
    heightEyeFt = df.loc[index]['heightEyeFt']
    return celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt

def read_lop_expected (df, index):
    LHA = readAngle(df, index, 'LHA')
    Ho = readAngle(df, index, 'Ho')
    Hc = readAngle(df, index, 'Hc')
    a = df.loc[index]['a']
    ta = df.loc[index]['ta']
    Zn = df.loc[index]['Zn']

    return LHA, Ho, Hc, a, ta, Zn

def read_fix_input (df, index):
    dr_lat_1 = readAngle(df, index, 'dr_lat_1')
    dr_lon_1 = readAngle(df, index, 'dr_lon_1')
    Zn_1 = readAngle(df, index, 'Zn_1')
    a_1 = df.loc[index]['a_1']
    ta_1 = df.loc[index]['ta_1']

    dr_lat_2 = readAngle(df, index, 'dr_lat_2')
    dr_lon_2 = readAngle(df, index, 'dr_lon_2')
    Zn_2 = readAngle(df, index, 'Zn_2')
    a_2 = df.loc[index]['a_2']
    ta_2 = df.loc[index]['ta_2']
    
    return dr_lat_1, dr_lon_1, Zn_1, a_1, ta_1, dr_lat_2, dr_lon_2, Zn_2, a_2, ta_2

df_lop_input = pd.read_csv("lop_input.csv")
df_lop_expected = pd.read_csv("lop_expected.csv")
df_fix_input = pd.read_csv("fix_input.csv")

# Greenwich Hour Angle (GHA) and Local Hour Angle (LHA)

In [6]:
def compute_GHA_dec(celestial_obj, utc): 
    dt = datetime.strptime(utc, '%Y/%m/%d %H:%M:%S')
    t = ts.ut1(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
    position = earth.at(t).observe(celestial_obj)
    ra = position.apparent().radec(epoch='date')[0]
    dec = position.apparent().radec(epoch='date')[1]
    distance = position.apparent().distance()
    
    gha = (t.gast-ra.hours)*15
    
    if (gha < 0):
        gha += 360

    return gha, dec.degrees, distance.km   

def compute_LHA(GHA, dr_lon):
    # Compute LHA.
    if (dr_lon < 0):
        # Western hemisphere.  LHA = GHA - a_lon
        LHA = GHA - (dr_lon*W)
    else:
        # Easter hemisphere.  LHA = GHA + a_lon
        LHA = GHA + (dr_lon*E)
        
    if (LHA < 0):
        LHA += 360.0
    elif (LHA > 360.0):
        LHA -= 360.0
    return LHA

# Sextant Corrections

In [7]:
Off = 1
On = -1

LowerLimb_Sun = 0
UpperLimb_Sun = 1
LowerLimb_Moon = 2
UpperLimb_Moon = 3

#SFalmanac uses volumetric mean radius in SD calculations for some reason.
moon_equatorial_radius = 1738.1# equatorial radius of moon = 1738.1 km
#moon_volumetric_radius = 1737.4# volumetric mean radius of moon = 1737.4 km

earth_equatorial_radius = 6378.0 # equatorial radius of the earth = 6378.0 km
#earth_volumetric_radius = 6371.0 # volumetric mean radius of earth = 6371.0 km

sun_equatorial_radius = 695700 # equatorial and volumetric mean radius of sun is the same = 695700 km

def compute_dip_correction(heightEyeFt):
    dip = 0.97*sqrt(heightEyeFt)
    dip = normal_round(dip, 1)
    dip *= -1
    return dip

def compute_refraction(Ha):
    # Round refraction to tenth of minutes... always negative.
    refraction = 1.0/ tan(radians(Ha + (7.31/(Ha+4.4))))
    refraction = normal_round(refraction, 1)
    refraction *= -1
    
    return refraction

def compute_semi_diameter_correction(Ha, distance, limb):
    celestial_body_radius = 0

    if ((limb == LowerLimb_Sun) or (limb == UpperLimb_Sun)):
        celestial_body_radius = sun_equatorial_radius
    elif ((limb == LowerLimb_Moon) or (limb == UpperLimb_Moon)):
        celestial_body_radius = moon_equatorial_radius
        
    sd = degrees(asin(celestial_body_radius / distance))
   
    sd_minutes = normal_round(sd * 60, 1)
    if ((limb == UpperLimb_Sun) or (limb == UpperLimb_Moon)):
        sd_minutes *= -1
    
    return sd_minutes
    
def compute_parallax_correction(Ha, distance, limb, lat):
    #hp = degrees(asin(earth_equatorial_radius/distance)) 
    hp = asin(earth_equatorial_radius/distance) 
    sd_augmentation = 0
    
    if ((limb == LowerLimb_Moon) or (limb == UpperLimb_Moon)):
        hp_correction = hp * sin(radians(lat))**2 / 298.3
        hp -= hp_correction
        
        sd_augmentation = sin(radians(Ha)) * hp
        sd_augmentation = degrees(sd_augmentation)
        #print("sd_augmentation: {}".format(sd_augmentation))
        sd_augmentation = 0 # TODO Figure this out as it doesn't work currently.
        
    # Compute Parallax in Altitude
    hp = hp * cos(radians(Ha))
    hp = degrees(hp)
    hp_minutes = normal_round(hp * 60, 1)
    
    return hp_minutes, sd_augmentation


# Sight Reduction

In [8]:
def sight_reduction(dec, lat, LHA):
    dec_rads = radians(dec)
    lat_rads = radians(lat)
    LHA_rads = radians(LHA)
    
    if ((dec_rads < 0) and (lat_rads < 0)):
        # If dec and lat are same hemisphere (i.e. SAME), then both values should be positive.
        # here there are both south (so negative) take the absolute value so that they are both positive.
        dec_rads = abs(dec_rads)
        lat_rads = abs(lat_rads)
    elif ((dec_rads > 0) and (lat_rads < 0)):
        # if dec and lat different hemispheres (i.e. CONTRARY), then dec should be made negative.
        # here we have a south latitude and north declination, so shift the negative to dec.
        dec_rads = dec_rads *-1
        lat_rads = lat_rads *-1
    
    # Hc
    sin_Hc_rads = sin(lat_rads)*sin(dec_rads) + cos(lat_rads)*cos(dec_rads)*cos(LHA_rads) 
    Hc_rads = asin(sin_Hc_rads)
    Hc = degrees(Hc_rads)
    
    #Z       
    cos_Z_rads_num = sin(dec_rads) - sin(Hc_rads)*sin(lat_rads)
    cos_Z_rads_den = cos(Hc_rads) * cos(lat_rads) 
    cos_Z_rads = cos_Z_rads_num / cos_Z_rads_den
    
    # Clip values that are > 1 or < -1  (usually introduced due to rounding error)
    if (cos_Z_rads > 1):
        print("cos_Z_rads > 1 {}".format(cos_Z_rads))
        cos_Z_rads = 1
    elif (cos_Z_rads < -1):
        print("cos_Z_rads < 1 {}".format(cos_Z_rads))
        cos_Z_rads = -1
    
    Z = degrees(acos(cos_Z_rads))
    
    return Hc, Z

def compute_Zn(Z, lat, LHA):
    #Zn
    Zn = Z
    if (lat > 0):
        # Northern latitude.
        if (LHA < 180):
            Zn = 360 - Z
    else:
        # Souther latitude.
        if (LHA > 180):
            Zn = 180-Z
        else:
            Zn = 180+Z
            
    if (Zn == 360):
        Zn = 0
    return Zn

def compute_intercept(Ho, Hc):
    # return a in nautical miles (i.e. minutes of latitude *=60)
    a = normal_round(abs(Ho-Hc) * 60, 1)
    ta = "T" if (Ho > Hc) else "A"
    return a, ta

# Compute Line Of Position

In [9]:
def compute_line_of_position(celestial_obj_string, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt):
    #print("utc: {}".format(utc))
    #print("dr_lat: {}".format(angleToString(dr_lat)))
    #print("dr_lon: {}".format(angleToString(dr_lon)))
    #print(celestial_obj_string)
    
    dt = datetime.strptime(utc, '%Y/%m/%d %H:%M:%S')
    celestial_obj = None
    limb = None
    
    if (celestial_obj_string == "Sun-LL"):
        celestial_obj = planets['Sun']
        limb = LowerLimb_Sun
    elif (celestial_obj_string == "Sun-UL"):
        celestial_obj = planets['Sun']
        limb = UpperLimb_Sun
    elif (celestial_obj_string == "Moon-LL"):
        celestial_obj = planets['Moon']
        limb = LowerLimb_Moon
    elif (celestial_obj_string == "Moon-UL"):
        celestial_obj = planets['Moon']
        limb = UpperLimb_Moon
    elif (celestial_obj_string == "Venus"):
        celestial_obj = planets['Venus']
    elif (celestial_obj_string == "Jupiter"):
        celestial_obj = planets['JUPITER BARYCENTER']
    else:
        celestial_obj = Star.from_dataframe(stars_df.loc[star_dictionary[celestial_obj_string]])
    
    GHA, dec, distance = compute_GHA_dec(celestial_obj, utc)
    
    print("GHA: {}".format(angleToString(GHA)))
    print("dec: {}".format(angleToString(dec)))
    print("distance: {}".format(distance))
    
    # First correct for index and DIP
    dip = compute_dip_correction(heightEyeFt) 
    Ha = Hs + ic + createAngle(0, dip, None)
    print("Hs: {}".format(angleToString(Hs)))
    print("ic: {}".format(angleToString(ic)))
    print("heightEyeFt: {}; dip (minutes): {}".format(heightEyeFt, dip)) 
    print("Ha: {}".format(angleToString(Ha)))
    
    # Second correct for refraction (returned in minutes)
    refraction_minutes = compute_refraction(Ha)
    print("refraction (minutes): {}".format(refraction_minutes))
    Ha = Ha + refraction_minutes / 60.0
    
    # Third correct for Semi-diameter (SD) and Horozontal Parallax
    sd_minutes = compute_semi_diameter_correction(Ha, distance, limb)
    hp_minutes, sd_augmentation = compute_parallax_correction(Ha, distance, limb, dr_lat)    
    print("SD (minutes): {}".format(sd_minutes))
    print("HP (minutes): {}".format(hp_minutes))
    
    # Finally can compute Ho.
    Ho = Ha + (sd_minutes + sd_augmentation + hp_minutes) / 60.0
    print("Ho: {}".format(angleToString(Ho)))

    LHA = compute_LHA(dr_lon, GHA)
    #print("LHA: {}".format(angleToString(LHA)))

    Hc, Z = sight_reduction(dec, dr_lat, LHA)
    Zn = compute_Zn(Z, dr_lat, LHA)
    print("Hc: {}".format(angleToString(Hc)))
    print("Z: {}".format(angleToString(Z)))
    print("Zn: {}".format(angleToString(Zn)))

    a, ta = compute_intercept(Ho, Hc)
    print("Azimuth: {} {}; Intercept: {}".format(angleToString(Zn), ta, a))
    
    return Ho, LHA, Hc, Zn, a, ta

# Compute Fix

In [10]:
nm_to_km = 1.852

def compute_towards_away_bearing(Zn, ta):
    if (ta == "A"):
        Zn += pi
    if (Zn >= 2*pi):
        Zn -= 2*pi
    return Zn

def sum_angles (angle1_rads, angle_2_rads):
    angle_sum = angle1_rads + angle_2_rads
    
    if (angle_sum > 2*pi):
        angle_sum -= 2*pi
    elif (angle_sum < 0):
        angle_sum += 2*pi
        
    return angle_sum

# Compute distance between two points (haversize formula)
def compute_distance (lat_1, lon_1, lat_2, lon_2):
    delta_lat = lat_2 - lat_1
    delta_lon = lon_2 - lon_1
    
    a = sin(delta_lat/2)**2 + cos(lat_1) * cos(lat_2) * sin(delta_lon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return earth_equatorial_radius * c

# Given a start point, initial bearing, and distance, this will calculate the destination point.
# parameters and return in radians.
def compute_destination (lat, lon, bearing, distance):
    distance_km = distance * nm_to_km
    ang_dist = distance_km / earth_equatorial_radius    
    
    dest_lat = asin(sin(lat) * cos(ang_dist) + cos(lat) * sin(ang_dist) * cos(bearing))
    
    dest_lon_delta = atan2(sin(bearing) * sin(ang_dist) * cos(lat), cos(ang_dist) - sin(lat) * sin(dest_lat))
    dest_lon = lon + dest_lon_delta
    
    return dest_lat, dest_lon

# Compute the intersection of two paths given start points and bearings
# parameters and return in radians.
def compute_intersection (lat_1, lon_1, bearing_1, lat_2, lon_2, bearing_2):
    delta_lat = lat_2 - lat_1
    delta_lon = lon_2 - lon_1
    ang_dist_1_2 = 2 * asin(sqrt(sin(delta_lat/2)**2 + cos(lat_1) * cos(lat_2) * sin(delta_lon/2)**2))
    
    theta_a = acos((sin(lat_2)-sin(lat_1)*cos(ang_dist_1_2)) / (sin(ang_dist_1_2)*cos(lat_1)))
    theta_b = acos((sin(lat_1)-sin(lat_2)*cos(ang_dist_1_2)) / (sin(ang_dist_1_2)*cos(lat_2)))
    
    theta_1_2 = 0
    theta_2_1 = 0
    
    if (sin(lon_2-lon_1) > 0):
        #print("theta_a")
        theta_1_2 = theta_a
        theta_2_1 = 2*pi - theta_b
    else:
        #print("theta_b")
        theta_1_2 = 2*pi - theta_a
        theta_2_1 = theta_b
        
    alpha_1 = bearing_1 - theta_1_2
    alpha_2 = theta_2_1 - bearing_2
    
    valid = True
    
    if (sin(alpha_1) == 0 and sin(alpha_2) == 0):
        #infinite solutions
        #print("Infinite solutions")
        valid = False
    elif (sin(alpha_1) * sin (alpha_2) < 0):
        #ambiguous solution
        #print("Ambiguous solution")
        valid = False
    
    if (valid == True):
        alpha_3 = acos(-cos(alpha_1)*cos(alpha_2) + sin(alpha_1) * sin(alpha_2) * cos(ang_dist_1_2))
        ang_dist_1_3 = atan2(sin(ang_dist_1_2) * sin(alpha_1) * sin(alpha_2), cos(alpha_2) + cos(alpha_1) * cos(alpha_3))

        lat_intersection = asin(sin(lat_1)*cos(ang_dist_1_3) + cos(lat_1)*sin(ang_dist_1_3)*cos(bearing_1))

        delta_lon_intersection = atan2(sin(bearing_1)*sin(ang_dist_1_3)*cos(lat_1), cos(ang_dist_1_3)-sin(lat_1)*sin(lat_intersection))

        lon_intersection = lon_1 + delta_lon_intersection
    else:
        lat_intersection = 0
        lon_intersection = 0
        
    if (lon_intersection < -pi):
        lon_intersection +=2*pi
        
    return lat_intersection, lon_intersection, valid
    
def compute_fix (lat_1, lon_1, Zn_1, a_1, ta_1, lat_2, lon_2, Zn_2, a_2, ta_2):
    lat_1_rads = radians(lat_1)
    lon_1_rads = radians(lon_1)
    bearing_1_rads = compute_towards_away_bearing(radians(Zn_1), ta_1)
    
    lat_2_rads = radians(lat_2)
    lon_2_rads = radians(lon_2)
    bearing_2_rads = compute_towards_away_bearing(radians(Zn_2), ta_2)
    
    lop_lat_1, lop_lon_1 = compute_destination (lat_1_rads, lon_1_rads, bearing_1_rads, a_1)
    lop_lat_2, lop_lon_2 = compute_destination (lat_2_rads, lon_2_rads, bearing_2_rads, a_2)
    
    bearing_options = [
        [sum_angles(bearing_1_rads, pi/2), sum_angles(bearing_2_rads, pi/2)],
        [sum_angles(bearing_1_rads, pi/2), sum_angles(bearing_2_rads, -pi/2)],
        [sum_angles(bearing_1_rads, -pi/2), sum_angles(bearing_2_rads, pi/2)],
        [sum_angles(bearing_1_rads, -pi/2), sum_angles(bearing_2_rads, -pi/2)],
    ]
    
    fix_lat = 0
    fix_lon = 0
    found = False
    
    for i in range(len(bearing_options)):
        if (found == False):
            lop_bearing_1, lop_bearing_2 = bearing_options[i]
            fix_lat, fix_lon, valid = compute_intersection (lop_lat_1, lop_lon_1, lop_bearing_1, lop_lat_2, lop_lon_2, lop_bearing_2)
            #print("valid: {}; fix_lat: {}; fix_lon: {}".format(valid, angleToString(degrees(fix_lat)), angleToString(degrees(fix_lon))))

            distance_km = compute_distance(lat_1_rads, lon_1_rads, fix_lat, fix_lon)
            #print("distance: {}".format(distance_km))

            if ((valid == True) and (distance_km < 5000)):
                #print("found fix")
                found = True
        
    
    return degrees(fix_lat), degrees(fix_lon), found
    
    


# Verify Line of Position

In [11]:
threshold = createAngle(0, 0.2, None)
a_threshold = 0.21 # need to revisit moon to lower this threshold further.
Zn_threshold = createAngle(0, 3, None) # need to revisit astron site to lower this threshold further.

success = True

for i in range (len(df_lop_input.index)):
    celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = read_lop_input (df_lop_input, i)
    Ho, LHA, Hc, Zn, a, ta = compute_line_of_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)
    
    LHA_expected, Ho_expected, Hc_expected, a_expected, ta_expected, Zn_expected = read_lop_expected (df_lop_expected, i)
    
    print ("SIGHTING for {} at {}: index: {}".format(celestial_object, utc, i))
    
    LHA_delta = diffAngle(LHA, LHA_expected, threshold)
    LHA_passed = (LHA_delta<threshold)
    print ("{}: LHA={}; delta={}".format("PASSED" if LHA_passed else "FAILED", angleToString(LHA), angleToStringDelta(LHA_delta)))
    
    Ho_delta = abs(Ho-Ho_expected)
    Ho_passed = (Ho_delta<threshold)
    print ("{}: Ho={}; delta={}".format("PASSED" if Ho_passed else "FAILED", angleToString(Ho), angleToStringDelta(Ho_delta)))
    
    Hc_delta = abs(Hc-Hc_expected)
    Hc_passed = (Hc_delta<threshold)
    print ("{}: Hc={}; delta={}".format("PASSED" if Hc_passed else "FAILED", angleToString(Hc), angleToStringDelta(Hc_delta)))
    
    a_delta = abs(a-a_expected)
    a_passed = ((a_delta<a_threshold) and (ta == ta_expected))
    print ("{}: a={} {}; delta={}".format("PASSED" if a_passed else "FAILED", a, ta, a_delta))
    if (ta != ta_expected):
        print("MISMATCH on T/A:  ta={}; ta_expected={}".format(ta, ta_expected))
   
    Zn_delta = diffAngle(Zn, Zn_expected, Zn_threshold)
    Zn_passed = (Zn_delta<Zn_threshold)
    print ("{}: Zn={}; delta={}".format("PASSED" if Zn_passed else "FAILED", angleToString(Zn), angleToStringDelta(Zn_delta)))
    
    print ("==========================")
    
    if (success == True):
        success = LHA_passed and Ho_passed and Hc_passed and a_passed and Zn_passed

    
print("Overall result: {}".format("PASSED" if (success==True) else "FAILED! Check above results."))



GHA: 164 degrees; 23.32 minutes
dec: 19 degrees; 34.36 minutes
distance: 151934492.69393405
Hs: 54 degrees; 5.0 minutes
ic: 0 degrees; 2.0 minutes
heightEyeFt: 9; dip (minutes): -2.9
Ha: 54 degrees; 0.1 minutes
refraction (minutes): -0.7
SD (minutes): 15.7
HP (minutes): 0.1
Ho: 54 degrees; 15.2 minutes
Hc: 53 degrees; 32.35 minutes
Z: 179 degrees; 49.41 minutes
Zn: 179 degrees; 49.41 minutes
Azimuth: 179 degrees; 49.41 minutes T; Intercept: 42.8
SIGHTING for Sun-LL at 1978/07/25 23:04:00: index: 0
PASSED: LHA=359 degrees; 53.32 minutes; delta=0.02 minutes
PASSED: Ho=54 degrees; 15.2 minutes; delta=0.0 minutes
PASSED: Hc=53 degrees; 32.35 minutes; delta=0.05 minutes
PASSED: a=42.8 T; delta=0.0
PASSED: Zn=179 degrees; 49.41 minutes; delta=1.41 minutes
GHA: 166 degrees; 16.65 minutes
dec: -12 degrees; 54.44 minutes
distance: 148648017.35720718
Hs: 87 degrees; 20.0 minutes
ic: 0 degrees; 2.0 minutes
heightEyeFt: 25; dip (minutes): -4.9
Ha: 87 degrees; 17.1 minutes
refraction (minutes): -0.

# Verify Fix Calculation

In [12]:
for i in range(len(df_fix_input.index)):
    dr_lat_1, dr_lon_1, Zn_1, a_1, ta_1, dr_lat_2, dr_lon_2, Zn_2, a_2, ta_2 = read_fix_input (df_fix_input, i)
    fix_lat, fix_lon, valid = compute_fix (dr_lat_1, dr_lon_1, Zn_1, a_1, ta_1, dr_lat_2, dr_lon_2, Zn_2, a_2, ta_2)
    print("valid: {}; fix_lat: {}; fix_lon: {}".format(valid, angleToString(fix_lat), angleToString(fix_lon)))

valid: True; fix_lat: 35 degrees; 34.58 minutes; fix_lon: -144 degrees; 32.22 minutes
valid: True; fix_lat: 34 degrees; 16.72 minutes; fix_lon: -146 degrees; 16.25 minutes
valid: True; fix_lat: 35 degrees; 20.99 minutes; fix_lon: -145 degrees; 35.3 minutes
valid: True; fix_lat: 34 degrees; 26.03 minutes; fix_lon: -146 degrees; 13.87 minutes
valid: True; fix_lat: 35 degrees; 27.69 minutes; fix_lon: -146 degrees; 18.71 minutes


# Illustration that deltas in DR result in a similar fix.

In [13]:
delta = [
    [-30, -30],
    [-30, 30],
    [30, 30],
    [30, -30]
]

celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = read_lop_input (df_lop_input, 12)
Ho_1, LHA_1, Hc_1, Zn_1, a_1, ta_1 = compute_line_of_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)

celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = read_lop_input (df_lop_input, 13)
Ho_2, LHA_2, Hc_2, Zn_2, a_2, ta_2 = compute_line_of_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)

fix_lat_org, fix_lon_org, valid = compute_fix (dr_lat, dr_lon, Zn_1, a_1, ta_1, dr_lat, dr_lon, Zn_2, a_2, ta_2)
print("dr_lat: {} dr_lon: {}".format(angleToString(dr_lat), angleToString(dr_lon)))
print("Original Fix is Lat:{}; Lon:{}".format(angleToString(fix_lat_org), angleToString(fix_lon_org)))


for i in range (len(delta)):
    lat_delta, lon_delta = delta[i]
    lat_delta = createAngle(0, lat_delta, None) + dr_lat
    lon_delta = createAngle(0, lon_delta, None) + dr_lon
    
    celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = read_lop_input (df_lop_input, 12)
    print("celestial_object 1: {}".format(celestial_object))
    Ho_1, LHA_1, Hc_1, Zn_1, a_1, ta_1 = compute_line_of_position (celestial_object, utc, lat_delta, lon_delta, Hs, ic, heightEyeFt)

    celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = read_lop_input (df_lop_input, 13)
    print("celestial_object 2: {}".format(celestial_object))
    Ho_2, LHA_2, Hc_2, Zn_2, a_2, ta_2 = compute_line_of_position (celestial_object, utc, lat_delta, lon_delta, Hs, ic, heightEyeFt)

    fix_lat, fix_lon, valid = compute_fix (lat_delta, lon_delta, Zn_1, a_1, ta_1, lat_delta, lon_delta, Zn_2, a_2, ta_2)
    print("Variant: {} =================================".format(i))
    print("a_lat: {} a_lon: {}".format(angleToString(lat_delta), angleToString(lon_delta)))
    print("Fix is Lat:{}; Lon:{}".format(angleToString(fix_lat), angleToString(fix_lon)))
    print("Delta is Lat:{}; Lon:{}".format(angleToString(fix_lat-fix_lat_org), angleToString(fix_lon-fix_lon_org)))


GHA: 66 degrees; 55.19 minutes
dec: 8 degrees; 48.87 minutes
distance: 158695484120317.78
Hs: 30 degrees; 35.4 minutes
ic: 0 degrees; 2.0 minutes
heightEyeFt: 16; dip (minutes): -3.9
Ha: 30 degrees; 33.5 minutes
refraction (minutes): -1.7
SD (minutes): 0.0
HP (minutes): 0.0
Ho: 30 degrees; 31.8 minutes
Hc: 30 degrees; 31.89 minutes
Z: 109 degrees; 22.53 minutes
Zn: 109 degrees; 22.53 minutes
Azimuth: 109 degrees; 22.53 minutes A; Intercept: 0.1
GHA: 117 degrees; 19.98 minutes
dec: -26 degrees; 23.05 minutes
distance: 5714217650853814.0
Hs: 18 degrees; 54.3 minutes
ic: 0 degrees; 2.0 minutes
heightEyeFt: 16; dip (minutes): -3.9
Ha: 18 degrees; 52.4 minutes
refraction (minutes): -2.9
SD (minutes): 0.0
HP (minutes): 0.0
Ho: 18 degrees; 49.5 minutes
Hc: 18 degrees; 52.48 minutes
Z: 175 degrees; 21.67 minutes
Zn: 175 degrees; 21.67 minutes
Azimuth: 175 degrees; 21.67 minutes A; Intercept: 3.0
dr_lat: 44 degrees; 36.0 minutes dr_lon: -122 degrees; 14.0 minutes
Original Fix is Lat:44 degrees;

# Illustration that iterations on a far off DR result in a good fix

In [14]:
celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = read_lop_input (df_lop_input, 12)
Ho_1, LHA_1, Hc_1, Zn_1, a_1, ta_1 = compute_line_of_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)

celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = read_lop_input (df_lop_input, 13)
Ho_2, LHA_2, Hc_2, Zn_2, a_2, ta_2 = compute_line_of_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)

fix_lat_org, fix_lon_org, valid = compute_fix (dr_lat, dr_lon, Zn_1, a_1, ta_1, dr_lat, dr_lon, Zn_2, a_2, ta_2)
print("dr_lat: {} dr_lon: {}".format(angleToString(dr_lat), angleToString(dr_lon)))
print("Original Fix is Lat:{}; Lon:{}".format(angleToString(fix_lat_org), angleToString(fix_lon_org)))

delta_degrees = 10

lat_delta = createAngle(delta_degrees, 0, None) + dr_lat
lon_delta = createAngle(delta_degrees, 0, None) + dr_lon

iterations = 3

for i in range (iterations):
    celestial_object, utc, _, _, Hs, ic, heightEyeFt = read_lop_input (df_lop_input, 12)
    Ho_1, LHA_1, Hc_1, Zn_1, a_1, ta_1 = compute_line_of_position (celestial_object, utc, lat_delta, lon_delta, Hs, ic, heightEyeFt)

    celestial_object, utc, _, _, Hs, ic, heightEyeFt = read_lop_input (df_lop_input, 13)
    Ho_2, LHA_2, Hc_2, Zn_2, a_2, ta_2 = compute_line_of_position (celestial_object, utc, lat_delta, lon_delta, Hs, ic, heightEyeFt)

    fix_lat, fix_lon, valid = compute_fix (lat_delta, lon_delta, Zn_1, a_1, ta_1, lat_delta, lon_delta, Zn_2, a_2, ta_2)
    print("Iteration: {} =================================".format(i))
    print("a_lat: {} a_lon: {}".format(angleToString(lat_delta), angleToString(lon_delta)))
    print("Fix is Lat:{}; Lon:{}".format(angleToString(fix_lat), angleToString(fix_lon)))
    print("Delta is Lat:{}; Lon:{}".format(angleToString(fix_lat-fix_lat_org), angleToString(fix_lon-fix_lon_org)))
    lat_delta = fix_lat
    lon_delta = fix_lon


GHA: 66 degrees; 55.19 minutes
dec: 8 degrees; 48.87 minutes
distance: 158695484120317.78
Hs: 30 degrees; 35.4 minutes
ic: 0 degrees; 2.0 minutes
heightEyeFt: 16; dip (minutes): -3.9
Ha: 30 degrees; 33.5 minutes
refraction (minutes): -1.7
SD (minutes): 0.0
HP (minutes): 0.0
Ho: 30 degrees; 31.8 minutes
Hc: 30 degrees; 31.89 minutes
Z: 109 degrees; 22.53 minutes
Zn: 109 degrees; 22.53 minutes
Azimuth: 109 degrees; 22.53 minutes A; Intercept: 0.1
GHA: 117 degrees; 19.98 minutes
dec: -26 degrees; 23.05 minutes
distance: 5714217650853814.0
Hs: 18 degrees; 54.3 minutes
ic: 0 degrees; 2.0 minutes
heightEyeFt: 16; dip (minutes): -3.9
Ha: 18 degrees; 52.4 minutes
refraction (minutes): -2.9
SD (minutes): 0.0
HP (minutes): 0.0
Ho: 18 degrees; 49.5 minutes
Hc: 18 degrees; 52.48 minutes
Z: 175 degrees; 21.67 minutes
Zn: 175 degrees; 21.67 minutes
Azimuth: 175 degrees; 21.67 minutes A; Intercept: 3.0
dr_lat: 44 degrees; 36.0 minutes dr_lon: -122 degrees; 14.0 minutes
Original Fix is Lat:44 degrees;

In [15]:
celestial_object = "Moon-LL"
utc = "2022/12/03 21:02:38"
dr_lat = createAngle(19, 39.51, N)
dr_lon = createAngle(155, 59.77, W)
Hs = createAngle(45, 0.0, None)
ic = createAngle(0.0, 1, Off)
heightEyeFt = 10

Ho, LHA, Hc, Zn, a, ta = compute_line_of_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)

GHA: 7 degrees; 7.73 minutes
dec: 6 degrees; 52.84 minutes
distance: 386666.40149554657
Hs: 45 degrees; 0.0 minutes
ic: 0 degrees; 1.0 minutes
heightEyeFt: 10; dip (minutes): -3.1
Ha: 44 degrees; 57.9 minutes
refraction (minutes): -1.0
SD (minutes): 15.5
HP (minutes): 40.1
Ho: 45 degrees; 52.5 minutes
Hc: -49 degrees; 27.71 minutes
Z: 52 degrees; 9.7 minutes
Zn: 52 degrees; 9.7 minutes
Azimuth: 52 degrees; 9.7 minutes T; Intercept: 5720.2


In [22]:
from skyfield.almanac import find_discrete, risings_and_settings

lat = createAngle(19, 42.47, N)
lon = createAngle(155, 58.73, W)

home = wgs84.latlon(lat, lon)

home_at_eargh = earth + home

celestial_obj_string = "Hamal"
star = Star.from_dataframe(stars_df.loc[star_dictionary[celestial_obj_string]])

In [23]:
f = risings_and_settings(planets, star, home)

In [30]:
t0 = ts.utc(2022, 12, 5)
t1 = ts.utc(2022, 12, 6)

t, y = find_discrete(t0, t1, f)

for ti, yi in zip(t, y):
    print(ti.utc_iso(), 'Rise' if yi else 'Set')

2022-12-05T00:58:30Z Rise
2022-12-05T14:13:35Z Set


In [ ]:
dt_object = datetime.fromtimestamp(timestamp)